In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score


In [3]:
# -------------------------------------------------------------
# 1️⃣ Load Dataset
# -------------------------------------------------------------
file_path = "student_performance_dataset_20.csv"  # update path if needed
df = pd.read_csv(file_path)

# Use relevant features
X = df[["Study_Hours_per_Week", "Attendance_Rate", "Internal_Scores"]].values
y = np.where(df["Pass_Fail"] == "Pass", 1, -1)

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train-test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [4]:
# -------------------------------------------------------------
# 2️⃣ Define Polynomial Kernel (From Scratch)
# -------------------------------------------------------------
def polynomial_kernel(X1, X2, degree=3, c=1):
    """
    Computes Polynomial Kernel:
    K(x1, x2) = (x1 · x2 + c)^degree
    """
    return (np.dot(X1, X2.T) + c) ** degree


In [5]:
# -------------------------------------------------------------
# 3️⃣ Define Simplified SVM Class (From Scratch)
# -------------------------------------------------------------
class SVMPolynomialScratch:
    def __init__(self, C=1.0, degree=3, c=1, lr=0.001, epochs=500):
        self.C = C
        self.degree = degree
        self.c = c
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        n_samples = X.shape[0]
        # Precompute Kernel Matrix
        self.K = polynomial_kernel(X, X, self.degree, self.c)
        self.alpha = np.zeros(n_samples)

        # Gradient Descent Training
        for _ in range(self.epochs):
            for i in range(n_samples):
                condition = y[i] * (np.sum(self.alpha * y * self.K[:, i])) >= 1
                if condition:
                    # No misclassification → regularization only
                    self.alpha[i] -= self.lr * self.alpha[i]
                else:
                    # Misclassified → update with penalty
                    self.alpha[i] += self.lr * (self.C - self.alpha[i])

        # Identify support vectors
        idx = self.alpha > 1e-5
        self.support_vectors_ = X[idx]
        self.support_labels_ = y[idx]
        self.support_alphas_ = self.alpha[idx]

    def project(self, X):
        # Apply Polynomial Kernel to Support Vectors
        K = polynomial_kernel(X, self.support_vectors_, self.degree, self.c)
        return np.dot(K, self.support_alphas_ * self.support_labels_)

    def predict(self, X):
        return np.sign(self.project(X))


In [ ]:
# -------------------------------------------------------------
# 4️⃣ Train and Evaluate Model
# -------------------------------------------------------------
svm_poly = SVMPolynomialScratch(C=1.0, degree=3, lr=0.001, epochs=500)
svm_poly.fit(X_train, y_train)
y_pred = svm_poly.predict(X_test)

# Evaluation Metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

